<a href="https://colab.research.google.com/github/badoil/ML/blob/master/multi_head_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(axis=-1, keepdim=True)
        std = x.std(axis=-1, keepdim=True)
        return self.a_2 * (x-mean)/(std+eps) + self.b_2

In [ ]:
from copy import deepcopy

In [ ]:
def clones(module, N):
    #module 반복할 클래스, N 반복횟수
    return nn.ModuleList([deepcopy(module) for _ in range(N)])

In [ ]:
# attention
def attention(query, key, value, mask=None, dropout=None):
    # Q, K, V : (batch_size, num_head, seq_length, d_k)  d_k 엠베딩 사이즈, 4차원 텐서
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)    # mask가 0인 애들(값이 0인 애들)을 아주작은 수로 바꿈
    p_attn = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)

    return torch.matmul(p_attn, value), p_attn

In [ ]:
# multi-head attention
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):   # h헤드의개수(8), d_model 모델 차원(512)
        super(MultiHeadedAttention, self).__init__()

        self.d_k = d_model // h
        self.h = h
        self.linears = clone(nn.Linear(d_model, d_model), 4)  # 뉴럴네트워크 4개
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        # Q, K, V : (batch_size, seq_length, d_model)
        if mask is not None:
            mask = mask.unsqueeze(1)  # h 헤드가 추가되므로 한 차원 늘림
        nbatches = query.size(0)

        # Q, K, V tensor 만들기 and head 추가
        # 아웃풋 (n_batch, h, seq_length, d_k)  -> Q, k, V
        # 아래 코드는 각 헤드에 대한 Q,K,V 텐서
        query, key, value = l(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2) for l, x in zip(self.linears, (query, key, value)):

        # attention 구하기
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)

        # 헤드 갯수인 8개의 매트릭스를 하나로 일렬로 concat & linear transformation 통해서 최종 어텐션 스코어 계산
        # x: (batch_size, num_head, seq_length, d_k)
        # 최종적 tensor shape (batch_size, seq_length, num_head * d_k)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h*self.d_k)  # concat
        return self.linear[-1](x) # (batch_size, seq_length, d_model) d_model로 차원축소

In [ ]:
## position-wise feed-forward network
class PositionWiseFeedForwardNetworkclass(nn.Module):
    def __init__(self, d_model, ff_model, dropout=0.1):
        super(PositionWiseFeedForwardNetworkclass, self).__init__()
        self.w_1 = nn.Linear(d_model, ff_model)
        self.w_2 = nn.Linear(ff_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [ ]:
#sublayer connection
#1) layer norm
#2) residual connection

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(x, sublayer):
        return x + self.dropout(sublayer(x))  # sublayer: ex. mutiheaded attention(class), FFNN(class)